In [16]:
import os
import psycopg2 as ps
import pandas as pd

In [17]:
conn = ps.connect(
    host='localhost',
    port='5432',
    dbname='chinook',
    user='chinook',
    password='password')

# Revenue generated by the top 10 countries

In [18]:
df = pd.read_sql_query('''SELECT 
                       c.country,
                       COUNT(i.invoice_id) AS num_invoice,
                       SUM(i.total) AS revenue,
                       ROUND(AVG(i.total), 2) AS avg_value
                       FROM customer c
                       JOIN invoice i
                       ON c.customer_id = i.customer_id
                       GROUP BY c.country
                       ORDER BY revenue DESC
                       LIMIT 10''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_154468\3366080018.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''SELECT


,country,num_invoice,revenue,avg_value
0,USA,91,523.06,5.75
1,Canada,56,303.96,5.43
2,France,35,195.10,5.57
3,Brazil,35,190.10,5.43
4,Germany,28,156.48,5.59
5,United Kingdom,21,112.86,5.37
6,Czech Republic,14,90.24,6.45
7,Portugal,14,77.24,5.52
8,India,13,75.26,5.79
9,Chile,7,46.62,6.66


### Key Takeaways:
1. High-Spending Markets: The Czech Republic and Chile stand out for their high average invoice values despite fewer transactions, indicating a concentrated but potentially lucrative market.
2. Large Markets with Lower Average Spend: The USA and Canada generate high revenues through a large number of transactions, but with lower average spending per invoice.
3. Balanced Markets: Countries like France, Brazil, and Germany show a balance between the number of transactions and average spending, making them stable markets with room for growth.

# Top 10 artists in the USA based on the number of tracks sold and the revenue they generated

In [19]:
df = pd.read_sql_query('''SELECT
	ar.name,
	SUM(il.quantity) AS num_tracks,
  SUM(il.unit_price * il.quantity) AS revenue
FROM artist ar
JOIN album al
ON ar.artist_id = al.artist_id
JOIN track t
ON al.album_id = t.album_id
JOIN invoice_line il
ON t.track_id = il.track_id
JOIN invoice i
ON il.invoice_id = i.invoice_id
WHERE i.billing_country = 'USA'
GROUP BY ar.artist_id
ORDER BY num_tracks DESC
LIMIT 10''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_154468\1004581078.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''SELECT


,name,num_tracks,revenue
0,Iron Maiden,34,33.66
1,Metallica,27,26.73
2,U2,23,22.77
3,The Office,17,33.83
4,Deep Purple,15,14.85
5,Eric Clapton,14,13.86
6,Led Zeppelin,13,12.87
7,Amy Winehouse,13,12.87
8,Guns N' Roses,13,12.87
9,Tim Maia,12,11.88


### Key Takeaways:
1. Iron Maiden, Metallica, and U2 are the most popular in terms of both sales and revenue.
2. The Office has a lower number of tracks sold but generates the highest revenue per track, suggesting it might have premium content.
3. The remaining artists have similar performance metrics, with consistent but slightly lower sales and revenue figures.
### Potential Actions:
1. Promote High-Performing Artists: Focus on marketing and promotional efforts for top artists like Iron Maiden, Metallica, and U2 to further boost sales.
2. Leverage Premium Content: The Office’s high revenue per track indicates an opportunity to push premium content, which could be a strategy to increase overall revenue.
3. Explore Pricing Strategies: Analyzing the pricing strategies for these artists could provide insights into maximizing revenue, especially for those with lower track sales but higher revenue per track.

# Most popular music genre in each country by analyzing the number of tracks sold

In [20]:
df = pd.read_sql_query('''WITH unranked AS (
  SELECT
		i.billing_country AS country,
		g.name AS genre,
  	SUM(il.quantity) AS num_tracks
	FROM genre g
	JOIN track t ON g.genre_id = t.genre_id
	JOIN invoice_line il ON t.track_id = il.track_id
	JOIN invoice i ON il.invoice_id = i.invoice_id
	GROUP BY i.billing_country, g.name
),
ranked AS (
  SELECT
		*,
  	ROW_NUMBER() OVER(PARTITION BY country ORDER BY num_tracks DESC) AS rn
	FROM unranked
)

SELECT
	country,
  genre
FROM ranked
WHERE rn = 1
ORDER BY num_tracks DESC''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_154468\1901467184.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''WITH unranked AS (


,country,genre
0,USA,Rock
1,Canada,Rock
2,Brazil,Rock
3,France,Rock
4,Germany,Rock
5,United Kingdom,Rock
6,Portugal,Rock
7,India,Rock
8,Czech Republic,Rock
9,Spain,Rock


### Insights:
1. Global Dominance of Rock: Rock music is the most popular genre in almost all the countries listed, which may suggest that it has a broad international appeal.
2. Regional Preferences: The only exception in the dataset is Sweden, where Latin music takes the top spot. This could indicate a particular cultural or demographic trend that makes Latin music more popular there.

### Potential Actions:
1. Market Focus: Given the dominance of Rock, marketing efforts might focus on promoting Rock music, especially in the countries where it is already most popular.
2. Explore Regional Trends: In Sweden, Latin music’s popularity might indicate an opportunity to explore or expand into Latin music offerings, or to investigate why this genre is uniquely popular there.


#  List of tracks, along with their album titles and artist names, that have not been purchased in the last 6 months

In [21]:
df = pd.read_sql_query('''SELECT t.name, al.title, ar.name
FROM track t
JOIN invoice_line il ON t.track_id = il.track_id
JOIN invoice i ON il.invoice_id = i.invoice_id
JOIN album al ON t.album_id = al.album_id
JOIN artist ar ON al.artist_id = ar.artist_id
GROUP BY t.name, al.title, ar.name
HAVING MAX(i.invoice_date) < NOW() - INTERVAL '6' MONTH;''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_154468\184942104.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''SELECT t.name, al.title, ar.name


,name,title,name
0,"Stay (Faraway, So Close!)",Zooropa,U2
1,Pensamento,Acústico MTV [Live],Cidade Negra
2,"De Do Do Do, De Da Da Da",The Police Greatest Hits,The Police
3,I Can't Explain,20th Century Masters - The Millennium Collecti...,Scorpions
4,Porch,Ten,Pearl Jam
...,...,...,...
1160,Me Deixas Louca,Elis Regina-Minha História,Elis Regina
1161,You Know I'm No Good,Back to Black,Amy Winehouse
1162,Leave My Girl Alone,In Step,Stevie Ray Vaughan & Double Trouble
1163,Seu Balancê,Ao Vivo [IMPORT],Zeca Pagodinho


### Potential Actions:
1. Launch targeted marketing campaigns and promotions to increase visibility and encourage purchases of the tracks.
2. Collaborate with artists for new releases or exclusive content and send notifications to users about these tracks.
3. Review and adjust pricing strategies, improve track recommendations, and ensure availability across all relevant platforms.

# List of the top 10 customers by revenue, including their customer ID, name, country, number of invoices, and total revenue. This will help identify the most valuable customers based on their total spending.

In [24]:
df = pd.read_sql_query('''SELECT
    c.customer_id,
    CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
    c.country,
    COUNT(i.invoice_id) AS num_invoice,
    SUM(i.total) AS revenue 
FROM customer c
JOIN invoice i
ON c.customer_id = i.customer_id  -- Corrected join condition
GROUP BY c.customer_id
ORDER BY revenue DESC
LIMIT 10;''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_154468\4025800924.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''SELECT


,customer_id,customer_name,country,num_invoice,revenue
0,6,Helena Holý,Czech Republic,7,49.62
1,26,Richard Cunningham,USA,7,47.62
2,57,Luis Rojas,Chile,7,46.62
3,46,Hugh O'Reilly,Ireland,7,45.62
4,45,Ladislav Kovács,Hungary,7,45.62
5,28,Julia Barnett,USA,7,43.62
6,24,Frank Ralston,USA,7,43.62
7,37,Fynn Zimmermann,Germany,7,43.62
8,25,Victor Stevens,USA,7,42.62
9,7,Astrid Gruber,Austria,7,42.62


### Potential Actions:
1. Engage Top Customers: Develop targeted marketing campaigns or loyalty programs for these high-value customers to further increase their engagement and spending.
2. VIP program: Provide special offers or rewards based on their purchasing behavior and preferences. Consider offering exclusive content or discounts to retain these valuable customers and encourage continued purchases.
3. Customer Feedback: Reach out to these top customers for feedback to understand their preferences and improve your services or offerings.